In [24]:
from pathlib import Path
import logging

# set logging level to info
logging.basicConfig(level=logging.INFO)

# For a relatively small iid count, create sample inpput files.

iid_count = 10_000

root = Path(r'm:\projects\pedigree')

id_file = root / f'test_sp_{iid_count}.grm.id'
id_file.parent.mkdir(parents=True, exist_ok=True)

index_to_id = [f"fid{i+1} iid{i+1} {i+1}" for i in range(iid_count)]

if not id_file.exists():
    with open(id_file, 'w') as f:
        for i in range(iid_count):
            f.write(f"{index_to_id[i]}\n")

index_to_id[:5]


['fid1 iid1 1', 'fid2 iid2 2', 'fid3 iid3 3', 'fid4 iid4 4', 'fid5 iid5 5']

In [17]:
# make a random iid_count x iid_count matrix with values between 0 and 1. It should be symmetric. Values on the diagonal should be 1.
import numpy as np

rng = np.random.default_rng(seed=0)

# create a random matrix of size iid_count x iid_count
sp = rng.random((iid_count, iid_count))/2

# make the matrix symmetric
sp = sp + sp.T - np.diag(sp.diagonal())
np.fill_diagonal(sp, 1)

sp[:5, :5]

array([[1.        , 0.41889681, 0.49668756, 0.278465  , 0.43328766],
       [0.41889681, 1.        , 0.62663821, 0.71354034, 0.43765316],
       [0.49668756, 0.62663821, 1.        , 0.63003756, 0.61090471],
       [0.278465  , 0.71354034, 0.63003756, 1.        , 0.17183549],
       [0.43328766, 0.43765316, 0.61090471, 0.17183549, 1.        ]])

In [28]:
sp_file = root / f'test_sp_{iid_count}.grm.sp'

if not sp_file.exists():
    with open(sp_file, 'w') as f:
        for i in range(iid_count):
            if i % 1000 == 0:
                logging.info(f"{i:_} of {iid_count:_}")
            for j in range(i, iid_count): # cmk ok to assume sp is symmetric?
               # print just 3 digits after the decimal point
               f.write(f"{i} {j} {sp[i,j]:.3f}\n")

INFO:root:0 of 10_000
INFO:root:1_000 of 10_000
INFO:root:2_000 of 10_000
INFO:root:3_000 of 10_000
INFO:root:4_000 of 10_000
INFO:root:5_000 of 10_000
INFO:root:6_000 of 10_000
INFO:root:7_000 of 10_000
INFO:root:8_000 of 10_000
INFO:root:9_000 of 10_000


In [32]:
# Create an in-memory kernel data object from the id and sp files.
from pysnptools.kernelreader import KernelData, KernelNpz

def read_grm(sp_file, save=False):
    sp_file = Path(sp_file)
    id_file = sp_file.with_suffix('.id')
    previous_sort = -1
    iid_list = []
    with open(id_file, 'r') as f:
        for line in f:
            fid, iid, sorter = line.split()
            sorter = int(sorter)
            assert sorter > previous_sort, "The .id file must be sorted by sort column"
            previous_sort = sorter
            iid_list.append([fid, iid])
    logging.info(f"Read {len(iid_list)} ids from {id_file}")

    val = np.zeros((len(iid_list), len(iid_list)))

    with open(sp_file, 'r') as f:
        for line_index, line in enumerate(f):
            if line_index % 1_000_000 == 0:
                logging.info(f"{line_index:_} of {int(iid_count*iid_count/2):_}")
            i, j, value = line.split()
            i = int(i)
            j = int(j)
            value = float(value)
            val[i, j] = value
            val[j, i] = value

    kernel_data = KernelData(iid=iid_list, val=val)
    if save:
        KernelNpz.write(str(sp_file.with_suffix('.kernel.npz')), kernel_data)
    return kernel_data

    

kd = read_grm(root / f'test_sp_{iid_count}.grm.sp', save=True)
kd.val[:5, :5]



INFO:root:Read 10000 ids from m:\projects\pedigree\test_sp_10000.grm.id
INFO:root:0 of 50_000_000
INFO:root:1_000_000 of 50_000_000
INFO:root:2_000_000 of 50_000_000
INFO:root:3_000_000 of 50_000_000
INFO:root:4_000_000 of 50_000_000
INFO:root:5_000_000 of 50_000_000
INFO:root:6_000_000 of 50_000_000
INFO:root:7_000_000 of 50_000_000
INFO:root:8_000_000 of 50_000_000
INFO:root:9_000_000 of 50_000_000
INFO:root:10_000_000 of 50_000_000
INFO:root:11_000_000 of 50_000_000
INFO:root:12_000_000 of 50_000_000
INFO:root:13_000_000 of 50_000_000
INFO:root:14_000_000 of 50_000_000
INFO:root:15_000_000 of 50_000_000
INFO:root:16_000_000 of 50_000_000
INFO:root:17_000_000 of 50_000_000
INFO:root:18_000_000 of 50_000_000
INFO:root:19_000_000 of 50_000_000
INFO:root:20_000_000 of 50_000_000
INFO:root:21_000_000 of 50_000_000
INFO:root:22_000_000 of 50_000_000
INFO:root:23_000_000 of 50_000_000
INFO:root:24_000_000 of 50_000_000
INFO:root:25_000_000 of 50_000_000
INFO:root:26_000_000 of 50_000_000
I

array([[1.   , 0.419, 0.497, 0.278, 0.433],
       [0.419, 1.   , 0.627, 0.714, 0.438],
       [0.497, 0.627, 1.   , 0.63 , 0.611],
       [0.278, 0.714, 0.63 , 1.   , 0.172],
       [0.433, 0.438, 0.611, 0.172, 1.   ]])

In [33]:
kr2 = KernelNpz(root / f'test_sp_{iid_count}.grm.kernel.npz').read()
kr2.val[:5, :5]

array([[1.   , 0.419, 0.497, 0.278, 0.433],
       [0.419, 1.   , 0.627, 0.714, 0.438],
       [0.497, 0.627, 1.   , 0.63 , 0.611],
       [0.278, 0.714, 0.63 , 1.   , 0.172],
       [0.433, 0.438, 0.611, 0.172, 1.   ]])